In [1]:
import pandas as pd
import numpy as np
import imageio
import matplotlib.pyplot as plt
from tensorflow.keras.backend import expand_dims
from tensorflow.keras.preprocessing import image_dataset_from_directory

2021-09-01 16:22:57.142135: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-01 16:22:57.143932: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

In [3]:
datagen = ImageDataGenerator(
    rescale = 1./255, # need to remember we have normalised here so don't do it within neural networks
    rotation_range = 5,
    zoom_range = (0.90, 0.90),
    brightness_range = (0.95, 0.95),
    horizontal_flip = True,
    vertical_flip = True,
    data_format = 'channels_last',
    validation_split = 0.2,
    dtype = tf.bfloat16
)

In [11]:
train_generator = datagen.flow_from_directory(
    '/home/iman/code/katarzyna-kupczyk/alzheimers_final_project/raw_data/AlzheimersDataset/train',
    target_size = (208, 176),
    batch_size = (32),
    class_mode = 'categorical',
    shuffle = True,
    subset = 'training',
    seed = 123
)

Found 4098 images belonging to 4 classes.


In [12]:
validation_generator = datagen.flow_from_directory(
    '/home/iman/code/katarzyna-kupczyk/alzheimers_final_project/raw_data/AlzheimersDataset/train',
    target_size = (208, 176),
    batch_size = (32),
    class_mode = 'categorical',
    shuffle = True,
    subset = 'validation',
    seed = 123
)


Found 1023 images belonging to 4 classes.


In [13]:

model = Sequential()
model.add(layers.Conv2D(16, (3,3), input_shape=(208, 176, 3), padding='same', activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Conv2D(32, (2,2), padding='same', activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2,2))) 
model.add(layers.Flatten())
model.add(layers.Dense(50, activation='relu')) # intermediate layer
model.add(layers.Dense(4, activation='softmax'))
model.summary()



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 208, 176, 16)      448       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 104, 88, 16)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 104, 88, 32)       2080      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 52, 44, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 73216)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                3660850   
_________________________________________________________________
dense_3 (Dense)              (None, 4)                

In [14]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='AUC')

In [15]:
es = EarlyStopping(patience=5, restore_best_weights=True)

In [16]:
model.fit(train_generator, validation_data=validation_generator, epochs=50, callbacks=[es], verbose=1)

Epoch 1/50
129/129 [==============================] - 144s 1s/step - loss: 1.0140 - auc: 0.8045 - val_loss: 1.1015 - val_auc: 0.7603
Epoch 2/50
129/129 [==============================] - 115s 894ms/step - loss: 0.8808 - auc: 0.8564 - val_loss: 1.1037 - val_auc: 0.7733
Epoch 3/50
129/129 [==============================] - 128s 992ms/step - loss: 0.8255 - auc: 0.8763 - val_loss: 1.2374 - val_auc: 0.7291
Epoch 4/50
129/129 [==============================] - 95s 737ms/step - loss: 0.7616 - auc: 0.8949 - val_loss: 1.1109 - val_auc: 0.7928
Epoch 5/50
129/129 [==============================] - 84s 645ms/step - loss: 0.7197 - auc: 0.9073 - val_loss: 1.0500 - val_auc: 0.7953
Epoch 6/50
129/129 [==============================] - 90s 697ms/step - loss: 0.6546 - auc: 0.9232 - val_loss: 1.1243 - val_auc: 0.7861
Epoch 7/50
129/129 [==============================] - 102s 789ms/step - loss: 0.6179 - auc: 0.9309 - val_loss: 1.2086 - val_auc: 0.8021
Epoch 8/50
129/129 [==============================] - 

In [19]:
model.evaluate(validation_generator)


32/32 [==============================] - 16s 469ms/step - loss: 1.0514 - auc: 0.7946


[1.0514230728149414, 0.7946208119392395]